In [1]:
# Script Name: Option_1 (Delta Neutral, Dynamic Spot-Futures Fill)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import json
import os

# API Keys (Bybit and Bitget only for this trial)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'

# CONFIGS
SPREAD_THRESHOLD = 0.4 / 100
TRADE_USDT_SIZE = 10
ADD_POSITION_USDT = 15
FUTURE_RATIO = 1.0
FUTURE_LEVERAGE = 1
CHECK_INTERVAL = 1
MIN_VOLUME = 50000
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADE_FILE = 'open_trades.json'

TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
futures_markets_cache = {}
open_trades = {}
exchanges = {}

# INIT EXCHANGES
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'defaultType': 'spot'}
})
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'defaultType': 'spot'}
})
exchanges = {'Bybit': bybit, 'Bitget': bitget}


In [ ]:
# STORAGE

def load_open_trades():
    if os.path.exists(TRADE_FILE):
        with open(TRADE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_open_trades(trades):
    with open(TRADE_FILE, 'w') as f:
        json.dump(trades, f, indent=2)

open_trades = load_open_trades()

# UTILS

def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last")
    except:
        return None

def get_best_bid_ask(exchange, symbol):
    ob = exchange.fetch_order_book(symbol)
    bid = ob['bids'][0][0] if ob['bids'] else None
    ask = ob['asks'][0][0] if ob['asks'] else None
    return bid, ask

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    return next((f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")), None)

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable = []
        futures_markets_cache[exchange_name] = [s for s in markets if ":USDT" in s or "-USDT-SWAP" in s]
        for s, m in markets.items():
            if exchange_name == "Bitget" and not m.get('spot'): continue
            if '/USDT' in s and ':USDT' not in s and '-USDT-SWAP' not in s:
                try:
                    vol = exchange.fetch_ticker(s).get('quoteVolume', 0)
                    if vol and vol >= MIN_VOLUME:
                        tradable.append(s)
                except: continue
        return tradable
    except: return []

def ask_user_permission(symbol, spread, net_profit):
    ans = input(f"\n🔍 Opportunity: {symbol} | Spread: {spread:.4%} | Net: {net_profit:.4%}. Execute? (yes/no): ").strip().lower()
    return ans == 'yes'

def execute_trade(exchange_name, symbol, usdt_size):
    try:
        exchange = exchanges[exchange_name]
        spot_price = get_price(exchange, symbol)
        if not spot_price:
            print(f"❌ No spot price for {symbol}")
            return

        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)
        if not futures_symbol:
            print(f"❌ No futures match for {symbol}")
            return

        amount = round(usdt_size / spot_price, 6)
        best_bid, _ = get_best_bid_ask(exchange, symbol)
        order = exchange.create_order(symbol=symbol, type='limit', side='buy', amount=amount, price=best_bid)

        wait_time, filled = 0, 0
        while wait_time < 15:
            try:
                order_info = exchange.fetch_order(order['id'], symbol)
                filled = order_info['filled']
                if filled >= amount * 0.98:
                    break
                new_bid, _ = get_best_bid_ask(exchange, symbol)
                if new_bid != best_bid:
                    best_bid = new_bid
                    exchange.cancel_order(order['id'], symbol)
                    order = exchange.create_order(symbol=symbol, type='limit', side='buy', amount=amount, price=best_bid)
            except Exception as e:
                print(f"⚠️ Spot order check error: {e}")
            time.sleep(1)
            wait_time += 1

        if filled == 0:
            try:
                exchange.cancel_order(order['id'], symbol)
            except: pass
            print("⚠️ Limit not filled. Using market order...")
            try:
                if exchange_name == 'Bitget':
                    order = exchange.create_order(symbol=symbol, type='market', side='buy', amount=None, params={"cost": usdt_size, "createMarketBuyOrderRequiresPrice": False})
                elif exchange_name == 'Bybit':
                    order = exchange.create_order(symbol=symbol, type='market', side='buy', amount=None, params={"cost": usdt_size})
                else:
                    order = exchange.create_order(symbol=symbol, type='market', side='buy', amount=amount)
                time.sleep(2)
                filled = order.get('filled', 0)
            except Exception as e:
                print(f"❌ Market order failed. Skipping futures leg. Reason: {e}")
                return

            if not filled or filled <= 0:
                print("❌ Market order filled 0. Skipping futures leg.")
                return

        if exchange_name == 'Bybit':
            fut = ccxt.bybit({'apiKey': BYBIT_API_KEY, 'secret': BYBIT_API_SECRET, 'options': {'defaultType': 'future'}})
        else:
            fut = ccxt.bitget({'apiKey': BITGET_API_KEY, 'secret': BITGET_API_SECRET, 'password': BITGET_API_PASSWORD, 'options': {'defaultType': 'swap'}})

        fut.load_markets()
        fut.set_leverage(FUTURE_LEVERAGE, futures_symbol)
        hedge_amount = filled * FUTURE_RATIO
        if hedge_amount <= 0:
            print("❌ Invalid hedge amount. Skipping futures.")
            return

        fut.create_order(symbol=futures_symbol, type='market', side='sell', amount=hedge_amount)

        trade_id = f"{symbol.replace('/', '')}_{int(time.time())}"
        open_trades[trade_id] = {
            "symbol": symbol,
            "exchange": exchange_name,
            "amount": filled,
            "hedge_amount": hedge_amount,
            "spot_price": spot_price,
            "futures_price": get_price(fut, futures_symbol),
            "entry_spread": (get_price(fut, futures_symbol) - spot_price) / spot_price,
            "timestamp": datetime.datetime.now().isoformat()
        }
        save_open_trades(open_trades)
        print(f"✅ Executed & saved: {trade_id}")

    except Exception as e:
        print(f"❌ Execution error: {e}")

# REMAINING CORE LOGIC (CHECK/PROCESS/MAIN) UNCHANGED

def check_arbitrage(exchange, exchange_name, symbol):
    spot_price = get_price(exchange, symbol)
    futures_symbol = find_futures_symbol(symbol, futures_markets_cache.get(exchange_name, []))
    if not futures_symbol:
        return
    futures_price = get_price(exchange, futures_symbol)
    if not spot_price or not futures_price:
        return

    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - TRADING_FEES[exchange_name] * 2
    if net_profit < SPREAD_THRESHOLD:
        return

    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
    print(f"🔎 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
    print(f"  ✅ Spot: {spot_price:.5f} | Futures: {futures_price:.5f}")
    print(f"  ✅ Spread: {spread:.4%} | Net: {net_profit:.4%}")

    for tid, t in open_trades.items():
        if t['symbol'] == symbol and t['exchange'] == exchange_name:
            if spread - t['entry_spread'] >= 0.005:
                print("🔁 Scaling opportunity.")
                if ask_user_permission(symbol, spread, net_profit):
                    execute_trade(exchange_name, symbol, ADD_POSITION_USDT)
            return

    if ask_user_permission(symbol, spread, net_profit):
        execute_trade(exchange_name, symbol, TRADE_USDT_SIZE)

def process_tokens(exchange, exchange_name, tokens):
    for token in tokens:
        check_arbitrage(exchange, exchange_name, token)

def main():
    print("\n--- Arbitrage Bot Started ---")
    while True:
        try:
            for name, ex in exchanges.items():
                tokens = get_tradable_tokens(ex, name)
                process_tokens(ex, name, tokens)
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Main loop error: {e}")
            time.sleep(5)

if __name__ == '__main__':
    main()



--- Arbitrage Bot Started ---
🔎 Arbitrage Opportunity on Bitget: PUMP/USDT | 2025-04-15 01:15:25 WIB
  ✅ Spot: 0.08996 | Futures: 0.09064
  ✅ Spread: 0.7559% | Net: 0.5959%
